<a href="https://colab.research.google.com/github/Hanzlazafar1/DIP_assignment_02/blob/main/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U pandas sentence-transformers tensorflow scikit-learn

In [10]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input , Bidirectional
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

In [2]:
# Install required packages
!pip install pandas

# Import required library
import pandas as pd

# Define a function to load dataset
def load_data(filepath):
    try:
        data = pd.read_csv(filepath, sep=';', names=["text", "label"])
        print(f"✅ Dataset loaded successfully from: {filepath}")
        print("📊 Sample data:")
        print(data.head())
        return data
    except Exception as e:
        print("❌ Failed to load dataset:", str(e))
        return None

# Load train and test datasets
train_path = '/content/train.txt'
test_path = '/content/test.txt'

train_df = load_data(train_path)
test_df = load_data(test_path)


✅ Dataset loaded successfully from: /content/train.txt
📊 Sample data:
                                                text    label
0                            i didnt feel humiliated  sadness
1  i can go from feeling so hopeless to so damned...  sadness
2   im grabbing a minute to post i feel greedy wrong    anger
3  i am ever feeling nostalgic about the fireplac...     love
4                               i am feeling grouchy    anger
✅ Dataset loaded successfully from: /content/test.txt
📊 Sample data:
                                                text    label
0  im feeling rather rotten so im not very ambiti...  sadness
1          im updating my blog because i feel shitty  sadness
2  i never make her separate from me because i do...  sadness
3  i left with my bouquet of red and yellow tulip...      joy
4    i was feeling a little vain when i did this one  sadness


In [5]:
# Load the latest embedding model from Hugging Face
print("⏳ Loading embedding model: jinaai/jina-embeddings-v2-base-en")
embedder = SentenceTransformer("jinaai/jina-embeddings-v2-base-en", trust_remote_code=True)
print("✅ Embedding model loaded.")

# Generate embeddings
print("🔄 Generating embeddings for train data...")
X_train = embedder.encode(train_df['text'].tolist(), convert_to_numpy=True)
print("✅ Train embeddings generated.")

print("🔄 Generating embeddings for test data...")
X_test = embedder.encode(test_df['text'].tolist(), convert_to_numpy=True)
print("✅ Test embeddings generated.")

⏳ Loading embedding model: jinaai/jina-embeddings-v2-base-en


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.3k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Embedding model loaded.
🔄 Generating embeddings for train data...
✅ Train embeddings generated.
🔄 Generating embeddings for test data...
✅ Test embeddings generated.


In [6]:
# Check shape of the embeddings
print("📐 Shape of X_train:", X_train.shape)  # (num_samples, embedding_dim)
print("📐 Shape of X_test:", X_test.shape)

# Check the type of embeddings
print("🔎 Type of X_train:", type(X_train))

# Print a sample embedding (first sentence vector)
print("📌 Sample embedding for first training example:")
print(X_train[0])

# Print the number of features (i.e., embedding dimensions)
print("📏 Embedding dimension:", X_train.shape[1])


📐 Shape of X_train: (16000, 768)
📐 Shape of X_test: (2000, 768)
🔎 Type of X_train: <class 'numpy.ndarray'>
📌 Sample embedding for first training example:
[-3.95551510e-02 -3.95243280e-02  4.04326245e-02  2.53929868e-02
 -3.77594563e-03  9.51796363e-04  1.98288169e-02  1.26833189e-03
  6.27146438e-02 -2.61263996e-02 -2.17890143e-02 -3.17979269e-02
 -4.60970849e-02  3.41701992e-02 -3.28490585e-02  5.77557608e-02
  1.51179638e-02  1.25867371e-02 -6.84696389e-03  1.22331986e-02
 -3.74923972e-03 -2.02724431e-02 -6.76232530e-03 -1.47960531e-02
  1.78563222e-02  2.70991009e-02  2.05588620e-02 -1.29013862e-02
  2.46985536e-03  2.59236265e-02 -6.03874214e-03 -8.61906633e-03
 -4.82844450e-02  6.27625443e-04 -2.79434919e-02 -4.02739691e-03
 -3.53442505e-02  1.22362627e-02  3.22599486e-02  3.75985056e-02
 -1.03123877e-02  1.00731337e-02  3.33123505e-02  3.79574932e-02
 -4.95929122e-02  9.16757062e-03 -1.32286986e-02  4.14201356e-02
 -3.13560246e-03 -4.08896878e-02 -3.25638205e-02 -2.15810370e-02
 

In [12]:
# Encode emotion labels
label_encoder = LabelEncoder()
y_train = to_categorical(label_encoder.fit_transform(train_df['label']))
y_test = to_categorical(label_encoder.transform(test_df['label']))

# Check shapes
print(f"✅ X_train shape: {X_train.shape}")
print(f"✅ y_train shape: {y_train.shape}")

# Define a powerful Feedforward Neural Network
model = Sequential([
    Input(shape=(X_train.shape[1],)),  # 768 features
    Dense(512, activation='relu'),
    Dropout(0.4),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(y_train.shape[1], activation='softmax')
])


✅ X_train shape: (16000, 1, 768)
✅ y_train shape: (16000, 6)


In [14]:
# If X_train shape is (16000, 1, 768), remove the middle dimension
if X_train.ndim == 3:
    X_train = X_train.squeeze(1)
    X_test = X_test.squeeze(1)


In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input , LSTM, Reshape
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential([
    Input(shape=(768,)),  # Input layer with embedding dimension of 768
    # Reshape the data to have 3 dimensions (samples, time_steps, features)
    # Here, we treat each embedding as a time_step, and there's only one time_step per embedding.
    # So, the shape will be (batch_size, 1, 768).
    Reshape((1, 768)),  # LSTM expects (batch_size, time_steps, features)

    # LSTM layer
    LSTM(128, return_sequences=False, activation='tanh'),  # LSTM layer with 128 units

    # Dense layers for classification
    Dense(512, activation='relu'),
    Dropout(0.4),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(y_train.shape[1], activation='softmax')  # Output layer
])
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Show summary
model.summary()

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    callbacks=[EarlyStopping(patience=5, restore_best_weights=True)]
)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape (Reshape)               │ (None, 1, 768)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       459,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 512)            │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 690,310 (2.63 MB)

 Trainable params: 690,310 (2.63 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.4687 - loss: 1.3757 - val_accuracy: 0.6375 - val_loss: 0.9763
Epoch 2/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.6414 - loss: 0.9689 - val_accuracy: 0.6963 - val_loss: 0.8330
Epoch 3/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.6868 - loss: 0.8597 - val_accuracy: 0.6600 - val_loss: 0.8884
Epoch 4/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.6967 - loss: 0.8103 - val_accuracy: 0.7038 - val_loss: 0.7815
Epoch 5/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7150 - loss: 0.7628 - val_accuracy: 0.7169 - val_loss: 0.7425
Epoch 6/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7220 - loss: 0.7197 - val_accuracy: 0.7113 - val_loss: 0.7862
Epoch 7/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.7318 - loss: 0.7058 - val_accuracy: 0.7306 - val_loss: 0.7125
Epoch 8/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.7442 - loss: 0.6744 - val_accuracy: 0

In [22]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)

# Print the results
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7340 - loss: 0.7192
Test Loss: 0.7075
Test Accuracy: 0.7355


In [23]:
# Save the trained model
model.save("emotion_lstm_model.h5")
print("✅ Model saved as 'emotion_lstm_model.h5'")

# Save the label encoder using joblib or pickle
import joblib
joblib.dump(label_encoder, "label_encoder.pkl")
print("✅ LabelEncoder saved as 'label_encoder.pkl'")


✅ Model saved as 'emotion_lstm_model.h5'
✅ LabelEncoder saved as 'label_encoder.pkl'


In [24]:
# Load necessary libraries
import numpy as np
import joblib
from tensorflow.keras.models import load_model
from sentence_transformers import SentenceTransformer

# Load saved model and label encoder
model = load_model("/content/emotion_lstm_model.h5")
label_encoder = joblib.load("/content/label_encoder.pkl")
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Predict function
def predict_emotion(text):
    # Get embedding
    embedding = embedder.encode([text])
    embedding = embedding.reshape((1, 1, embedding.shape[1]))  # (batch, timestep, features)

    # Predict
    prediction = model.predict(embedding)
    predicted_class = np.argmax(prediction)
    emotion = label_encoder.inverse_transform([predicted_class])[0]
    return emotion


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]